In [1]:
import clingo

from typing import Sequence

In [2]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [3]:
domain = """

contract("Instanz Tauschvertrag").

component("Lieferung Ration").
component("Rückabwicklung Ration").

component("Lieferung Brett").
component("Nachfrist Lieferung Brett").
component("Lieferung Brett cont.").

component("Lieferung Gemälde").

component("Lieferung Brett Austauschanspruch").
component("Lieferung Brett Differenzanspruch").

componentType("Sachleistung").
componentType("Nachfrist").
componentType("Geldleistung").
componentType("Austauschanspruch").
componentType("Differenzanspruch").

resource("Holz").
resource("Nahrung").
resource("Brett").
resource("Ration").
resource("Gemälde").
resource("Geld").

resourceType("Gattung").
resourceType("Spezies").
resourceType("Geld").

agent(bob).
agent(woody).

event(destroy(Resource)) :- resource(Resource).

action(gather("Holz")).
action(gather("Nahrung")).
action(process_to("Holz", "Brett")).
action(process_to("Nahrung", "Ration")).
action(give_to_because(Resource, Agent, Component)) :- agent(Agent), resource(Resource), component(Component).
action(accept_from_because(Resource, Agent, Component)) :- agent(Agent), resource(Resource), component(Component).

action(communicate_to_because("Am Vertrag festhalten", Agent, Component)) :- agent(Agent), component(Component).
action(communicate_to_because("Vom Vertrag zurücktreten", Agent, Component)) :- agent(Agent), component(Component).

"""
solve([domain]);

Answer 1: { action(gather("Holz")) action(gather("Nahrung")) action(process_to("Holz","Bretter")) action(process_to("Nahrung","Ration")) action(accept_from_because("Bretter",bob,"Lieferung Bretter")) action(accept_from_because("Bretter",bob,"Lieferung Bretter Austauschanspruch")) action(accept_from_because("Bretter",bob,"Lieferung Bretter Differenzanspruch")) action(accept_from_because("Bretter",bob,"Lieferung Bretter cont.")) action(accept_from_because("Bretter",bob,"Lieferung Gemälde")) action(accept_from_because("Bretter",bob,"Lieferung Ration")) action(accept_from_because("Bretter",bob,"Nachfrist Lieferung Bretter")) action(accept_from_because("Bretter",bob,"Rückabwicklung Ration")) action(accept_from_because("Bretter",woody,"Lieferung Bretter")) action(accept_from_because("Bretter",woody,"Lieferung Bretter Austauschanspruch")) action(accept_from_because("Bretter",woody,"Lieferung Bretter Differenzanspruch")) action(accept_from_because("Bretter",woody,"Lieferung Bretter cont.")) ac

In [5]:
trace_success = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 3).
occ_at(agent_does(bob, accept_from_because("Brett", woody, "Lieferung Brett")), 3).
"""
solve([trace_success]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,accept_from_because("Bretter",woody,"Lieferung Bretter")),3) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),3) }
SAT 1


In [6]:
trace_unmoeglich_glauebiger = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, destroy("Brett")), 3).

"""
solve([trace_unmoeglich_glauebiger]);

Answer 1: { occ_at(agent_does(bob,destroy("Bretter")),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) }
SAT 1


In [7]:
trace_GLE1_unmoeglich_glauebiger = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(agent_does(bob, destroy("Brett")), 3).

"""
solve([trace_GLE1_unmoeglich_glauebiger]);

Answer 1: { occ_at(agent_does(bob,destroy("Bretter")),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) }
SAT 1


In [8]:
trace_GLE2_unmoeglich_glauebiger = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, destroy("Brett")), 3).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 4).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 4).

"""
solve([trace_GLE2_unmoeglich_glauebiger]);

Answer 1: { occ_at(agent_does(bob,destroy("Bretter")),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),4) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),4) }
SAT 1


In [9]:
trace_unmoeglich_schuldner_festhalten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(woody, destroy("Brett")), 3).

occ_at(agent_does(bob, communicate_to_because("Am Vertrag festhalten", woody, "Lieferung Brett")), 5).

"""
solve([trace_unmoeglich_schuldner_festhalten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,communicate_to_because("Am Vertrag festhalten",woody,"Lieferung Bretter")),5) occ_at(agent_does(woody,destroy("Bretter")),3) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) }
SAT 1


In [10]:
trace_GLE1_unmoeglich_schuldner_festhalten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(agent_does(woody, destroy("Brett")), 3).

occ_at(agent_does(bob, communicate_to_because("Am Vertrag festhalten", woody, "Lieferung Brett")), 5).

"""
solve([trace_GLE1_unmoeglich_schuldner_festhalten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,communicate_to_because("Am Vertrag festhalten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,destroy("Bretter")),3) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) }
SAT 1


In [11]:
trace_GLE2_unmoeglich_schuldner_festhalten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(woody, destroy("Brett")), 3).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 4).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 4).

occ_at(agent_does(bob, communicate_to_because("Am Vertrag festhalten", woody, "Lieferung Brett")), 5).

"""
solve([trace_GLE2_unmoeglich_schuldner_festhalten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,communicate_to_because("Am Vertrag festhalten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),4) occ_at(agent_does(woody,destroy("Bretter")),3) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),4) }
SAT 1


In [12]:
trace_unmoeglich_schuldner_zuruecktreten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(woody, destroy("Brett")), 3).

occ_at(agent_does(bob, communicate_to_because("Vom Vertrag zurücktreten", woody, "Lieferung Brett")), 5).

"""
solve([trace_unmoeglich_schuldner_zuruecktreten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,communicate_to_because("Vom Vertrag zurücktreten",woody,"Lieferung Bretter")),5) occ_at(agent_does(woody,destroy("Bretter")),3) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) }
SAT 1


In [13]:
trace_GLE1_unmoeglich_schuldner_zuruecktreten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(agent_does(woody, destroy("Brett")), 3).

occ_at(agent_does(bob, communicate_to_because("Vom Vertrag zuruecktreten", woody, "Lieferung Brett")), 5).

"""
solve([trace_GLE1_unmoeglich_schuldner_zuruecktreten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,communicate_to_because("Vom Vertrag zuruecktreten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,destroy("Bretter")),3) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) }
SAT 1


In [14]:
trace_GLE2_unmoeglich_schuldner_zuruecktreten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(woody, destroy("Brett")), 3).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 4).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 4).

occ_at(agent_does(bob, communicate_to_because("Vom Vertrag zuruecktreten", woody, "Lieferung Brett")), 5).

"""
solve([trace_GLE2_unmoeglich_schuldner_zuruecktreten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,communicate_to_because("Vom Vertrag zuruecktreten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),4) occ_at(agent_does(woody,destroy("Bretter")),3) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),4) }
SAT 1


In [15]:
trace_unmoeglich_zufall_gattung = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(destroy("Brett"), 3).

"""
solve([trace_unmoeglich_zufall_gattung]);

Answer 1: { occ_at(destroy("Bretter"),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) }
SAT 1


In [16]:
trace_GLE1_unmoeglich_zufall_gattung = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(destroy("Brett"), 3).

"""
solve([trace_GLE1_unmoeglich_zufall_gattung]);

Answer 1: { occ_at(destroy("Bretter"),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) }
SAT 1


In [17]:
trace_GLE2_unmoeglich_zufall_gattung = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(destroy("Brett"), 3).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 4).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 4).

"""
solve([trace_GLE2_unmoeglich_zufall_gattung]);

Answer 1: { occ_at(destroy("Bretter"),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),4) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),4) }
SAT 1


In [18]:
trace_unmoeglich_zufall_spezies = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Gemälde")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).

occ_at(destroy("Gemälde"), 3).

"""
solve([trace_unmoeglich_zufall_spezies]);

Answer 1: { occ_at(destroy("Gemälde"),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(woody,gather("Gemälde")),0) }
SAT 1


In [19]:
trace_GLE1_unmoeglich_zufall_spezies = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Gemälde")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(destroy("Gemälde"), 3).

"""
solve([trace_GLE1_unmoeglich_zufall_spezies]);

Answer 1: { occ_at(destroy("Gemälde"),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,gather("Gemälde")),0) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) }
SAT 1


In [20]:
trace_GLE2_unmoeglich_zufall_spezies = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Gemälde")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(destroy("Gemälde"), 3).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 4).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 4).

"""
solve([trace_GLE2_unmoeglich_zufall_spezies]);

Answer 1: { occ_at(destroy("Gemälde"),3) occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),4) occ_at(agent_does(woody,gather("Gemälde")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),4) }
SAT 1


In [21]:
trace_verzug_schuldner_festhalten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, communicate_to_because("Am Vertrag festhalten", woody, "Lieferung Brett")), 5).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 6).
occ_at(agent_does(bob, accept_from_because("Brett", woody, "Lieferung Brett")), 6).

"""
solve([trace_verzug_schuldner_festhalten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,accept_from_because("Bretter",woody,"Lieferung Bretter")),6) occ_at(agent_does(bob,communicate_to_because("Am Vertrag festhalten",woody,"Lieferung Bretter")),5) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),6) }
SAT 1


In [22]:
trace_GLE1_verzug_schuldner_festhalten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(agent_does(bob, communicate_to_because("Am Vertrag festhalten", woody, "Lieferung Brett")), 5).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 6).
occ_at(agent_does(bob, accept_from_because("Brett", woody, "Lieferung Brett")), 6).

"""
solve([trace_GLE1_verzug_schuldner_festhalten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,accept_from_because("Bretter",woody,"Lieferung Bretter")),6) occ_at(agent_does(bob,communicate_to_because("Am Vertrag festhalten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),6) }
SAT 1


In [23]:
trace_GLE2_verzug_schuldner_festhalten = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, communicate_to_because("Am Vertrag festhalten", woody, "Lieferung Brett")), 5).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 6).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 6).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 7).
occ_at(agent_does(bob, accept_from_because("Brett", woody, "Lieferung Brett")), 7).

"""
solve([trace_GLE2_verzug_schuldner_festhalten]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,accept_from_because("Bretter",woody,"Lieferung Bretter")),7) occ_at(agent_does(bob,communicate_to_because("Am Vertrag festhalten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),6) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),6) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),7) }
SAT 1


In [24]:
trace_verzug_schuldner_zuruecktreten_success = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, communicate_to_because("Vom Vertrag zurücktreten", woody, "Lieferung Brett")), 5).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 6).
occ_at(agent_does(bob, accept_from_because("Brett", woody, "Lieferung Brett")), 6).

"""
solve([trace_verzug_schuldner_zuruecktreten_success]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,accept_from_because("Bretter",woody,"Lieferung Bretter")),6) occ_at(agent_does(bob,communicate_to_because("Vom Vertrag zurücktreten",woody,"Lieferung Bretter")),5) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),6) }
SAT 1


In [25]:
trace_GLE1_verzug_schuldner_zuruecktreten_success = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(agent_does(bob, communicate_to_because("Vom Vertrag zurücktreten", woody, "Lieferung Brett")), 5).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 6).
occ_at(agent_does(bob, accept_from_because("Brett", woody, "Lieferung Brett")), 6).

"""
solve([trace_GLE1_verzug_schuldner_zuruecktreten_success]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,accept_from_because("Bretter",woody,"Lieferung Bretter")),6) occ_at(agent_does(bob,communicate_to_because("Vom Vertrag zurücktreten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),6) }
SAT 1


In [26]:
trace_GLE2_verzug_schuldner_zuruecktreten_success = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, communicate_to_because("Vom Vertrag zurücktreten", woody, "Lieferung Brett")), 5).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 6).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 6).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 7).
occ_at(agent_does(bob, accept_from_because("Brett", woody, "Lieferung Brett")), 7).

"""
solve([trace_GLE2_verzug_schuldner_zuruecktreten_success]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,accept_from_because("Bretter",woody,"Lieferung Bretter")),7) occ_at(agent_does(bob,communicate_to_because("Vom Vertrag zurücktreten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),6) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),6) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),7) }
SAT 1


In [27]:
trace_verzug_schuldner_zuruecktreten_fail = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, communicate_to_because("Vom Vertrag zurücktreten", woody, "Lieferung Brett")), 5).

"""
solve([trace_verzug_schuldner_zuruecktreten_fail]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,communicate_to_because("Vom Vertrag zurücktreten",woody,"Lieferung Bretter")),5) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) }
SAT 1


In [28]:
trace_GLE_verzug_schuldner_zuruecktreten_fail = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(agent_does(bob, communicate_to_because("Vom Vertrag zurücktreten", woody, "Lieferung Brett")), 5).

"""
solve([trace_GLE_verzug_schuldner_zuruecktreten_fail]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,communicate_to_because("Vom Vertrag zurücktreten",woody,"Lieferung Bretter")),5) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) }
SAT 1


In [29]:
trace_verzug_glauebiger = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 3).

"""
solve([trace_verzug_glauebiger]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),3) }
SAT 1


In [30]:
trace_GLE1_verzug_glauebiger = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 2).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 2).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 3).

"""
solve([trace_GLE1_verzug_glauebiger]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),2) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),2) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),3) }
SAT 1


In [31]:
trace_GLE2_verzug_glauebiger = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(woody, give_to_because("Brett", bob, "Lieferung Brett")), 3).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 6).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 6).

"""
solve([trace_GLE2_verzug_glauebiger]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Rationen")),6) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Bretter")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Rationen")),6) occ_at(agent_does(woody,give_to_because("Bretter",bob,"Lieferung Bretter")),3) }
SAT 1
